# Master file Tomato - Testing the SSL Approach

Our team attempted a self-supervised algorithm for the tomato and potato models in attempt to increase model generalizability. 

# 1. Load the data



In [ ]:
# download the Kaggle dataset instructions here:
# https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

# configure path to Kaggle json file 
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

#download kaggle dataset
# comment out so i dont re-download everytime
# !kaggle datasets download -d emmarex/plantdisease

#unzip the Plan Village Dataset
#unzipping the zip files and deleting the zip files
# !unzip \*.zip  && rm *.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/1AE_ah6OZ6tuGxdM5Y8ousz-lKzgRa6TZ/Kaggle


# 2. Importing Libraries

In [ ]:
import random as random
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn import preprocessing
from scipy.ndimage import rotate
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers, Input
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from keras.regularizers import l2
%matplotlib inline

# 3. Data Preprocessing

In [ ]:
print(os.getcwd())
data_dir=r'/content/gdrive/My Drive/Kaggle/PlantVillage_Sorted/'
general_data_dir=r'/content/gdrive/My Drive/General_Validation_Datasets/'

/content/gdrive/.shortcut-targets-by-id/1AE_ah6OZ6tuGxdM5Y8ousz-lKzgRa6TZ/Kaggle


### Tomato Image Data Generator Classes

Only using 4 tomato classes here:

* Tomato_healthy
* Tomato_Leaf_Mold
* Tomato__Target_Spot
* Tomato__Tomato_mosaic_virus


In [ ]:
seed = 909 # (IMPORTANT) to input image and corresponding target with same augmentation parameter.
image_size = (250,250)
batch = 32
# the image hierarchy is important
# see: https://stackoverflow.com/questions/49535561/imagedatagenerator-on-a-folder

# kaggle --->PlantVillage folder
#               |
#               Development folder
#                   split the total amount of images into 80% development, 
#                   and this will be further split by dev_params into tran and val generators (in code)
#                         |___> all 9 classes exist in this folder with 80% of the total photos
#               |
#               |
#               Test folder
#                   split the total amount of images into 20% test
#                         |___> all 9 classes exist in this folder with 20% of the total photos

# General_Validation_Datasets
#               | Contains all classes with data found from general sources

# CREATE THE TRAIN/VAL GENERATORS HERE
# notice that i split the gen_params into 10% val 90% train 
dev_params = { "validation_split":0.1, "rescale":1.0/255,"featurewise_center":False,"samplewise_center":False,"featurewise_std_normalization":False,\
              "samplewise_std_normalization":False,"zca_whitening":False,"rotation_range":20,"width_shift_range":0.1,"height_shift_range":0.1,\
              "shear_range":0.2, "zoom_range":0.1,"horizontal_flip":True,"fill_mode":'nearest',\
               "cval": 0}

train_image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**dev_params) 
val_image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**dev_params) 

# use classes tag to only specify a certain class 
train_generator = train_image_datagen.flow_from_directory(data_dir+'development', subset = 'training', classes = [ 'Tomato_healthy', 'Tomato_Leaf_Mold', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus'],
                                                     batch_size = batch,seed=seed, target_size=image_size,color_mode='rgb',shuffle = True)
val_generator = val_image_datagen.flow_from_directory(data_dir+'development', subset = 'validation', classes = [ 'Tomato_healthy', 'Tomato_Leaf_Mold', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus'],
                                                     batch_size = batch,seed=seed, target_size=image_size,color_mode='rgb',shuffle = True)

# CREATE THE TEST GENERATOR HERE
test_dir = data_dir+'test' # This is the test folder!!
test_params = {"rescale":1.0/255,"featurewise_center":False,"samplewise_center":False,"featurewise_std_normalization":False,\
              "samplewise_std_normalization":False,"zca_whitening":False,"rotation_range":20,"width_shift_range":0.1,"height_shift_range":0.1,\
              "shear_range":0.2, "zoom_range":0.1,"horizontal_flip":True,"fill_mode":'nearest',\
               "cval": 0}
test_image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**test_params) 
# use classes tag to only specify a certain class 
test_generator = train_image_datagen.flow_from_directory(test_dir, classes = [ 'Tomato_healthy', 'Tomato_Leaf_Mold', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus'],
                                                         batch_size = batch,seed=seed, target_size=image_size,color_mode='rgb',shuffle = True)


# CREATE THE GENERAL TEST GENERATOR HERE (testing dataset outside of PlantVillage)
test = 'ON' # turning this test case on/off
if test is 'ON':
  general_test_params = {"rescale":1.0/255,"featurewise_center":False,"samplewise_center":False,"featurewise_std_normalization":False,\
                "samplewise_std_normalization":False,"zca_whitening":False,"rotation_range":10,"width_shift_range":0.1,"height_shift_range":0.1,\
                "shear_range":0.2, "zoom_range":0.1,"horizontal_flip":True,"fill_mode":'nearest',\
                "cval": 0}

  general_test_image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**test_params) 

  # use classes tag to only specify a certain class 
  general_test_generator = train_image_datagen.flow_from_directory(general_data_dir, classes = ['Tomato_healthy', 'Tomato_Leaf_Mold', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus'],
                                                          batch_size = batch,seed=seed, target_size=image_size,color_mode='rgb',shuffle = True)
else:
  pass

Found 3110 images belonging to 4 classes.
Found 344 images belonging to 4 classes.
Found 866 images belonging to 4 classes.
Found 16 images belonging to 4 classes.


# 4. SSL Pre-Text

Defining and training a model on a pretext task (i.e., image rotation)

## Defining the model

In [ ]:
def my_model_pretext(ishape = (250,250,3), k = 4, lr = 1e-4):
    model_input = tf.keras.layers.Input(shape = ishape)
    l1 = tf.keras.layers.Conv2D(48, (3,3), padding='same', activation='relu')(model_input)
    l2 = tf.keras.layers.Conv2D(48, (3,3), padding='same', activation='relu')(l1)
    l2_drop = tf.keras.layers.Dropout(0.25)(l2)
    l3 = tf.keras.layers.MaxPool2D((2,2))(l2_drop)
    l4 = tf.keras.layers.Conv2D(96, (3,3), padding='same', activation='relu')(l3)
    l5 = tf.keras.layers.Conv2D(96, (3,3), padding='same', activation='relu')(l4)
    l5_drop = tf.keras.layers.Dropout(0.25)(l5)
    flat = tf.keras.layers.Flatten()(l5_drop)
    model_pretext = tf.keras.models.Model(inputs = model_input, outputs = flat)
    out = tf.keras.layers.Dense(k,activation = 'softmax')(flat)
    model = tf.keras.models.Model(inputs = model_input, outputs = out)
    return model,model_pretext

In [ ]:
model2,model_pretext2 = my_model_pretext()
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics = ["accuracy"])
print(model_pretext2.summary())
print(model2.summary())

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 250, 250, 48)      1344      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 250, 250, 48)      20784     
_________________________________________________________________
dropout_10 (Dropout)         (None, 250, 250, 48)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 125, 125, 48)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 125, 125, 96)      41568     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 125, 125, 96)      830

## Define your callbacks (save your model, patience, etc.)

In [ ]:

model_name_pretext = "best_model_tomato_cnn_rot_pretext.h5"
model_name_pretext_no_top = "best_model_tomato_cnn_rot_pretext_no_top.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20)

monitor = tf.keras.callbacks.ModelCheckpoint(model_name_pretext, monitor='val_loss',\
                                             verbose=0,save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min')
# Learning rate schedule
def scheduler(epoch, lr):
    if epoch%10 == 0:
        lr = lr/2
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 0)

## Prepare your data to train pretext

In [ ]:
IMG_WIDTH = 250
IMG_HEIGHT = 250

# functions to resize, normalize and load the image data

def resize_image_PIL(image, width, height, channels=3):
  return np.resize(image, (height, width, channels))

def normalize_image(image):
  return image/255 

def load_plant_data_PIL(data_dir="/content/gdrive/My Drive/Kaggle/PlantVillage/All_Images/AllTomato", img_width=250, img_height=250):
  img_data = []

  for file in os.listdir(os.path.join(data_dir)):
    image_path = (os.path.join(data_dir, file))
    image = np.array(Image.open(image_path))
    image = resize_image_PIL(image=image, height=img_height, width=img_width)
    image = normalize_image(image)
    img_data.append(image)
  
  return np.asarray(img_data)

In [ ]:
# Preparing dataset

# X contains all tomato data for the 4 classes. Both the generalized and original plant village data. 
X = load_plant_data_PIL(img_width=IMG_WIDTH, img_height=IMG_HEIGHT) 

X_augmented = np.zeros((X.shape[0]*4,250,250,3))
X_augmented[::4] = X

# adding rotations to images
X_augmented[1::4] = rotate(X, angle = 90, axes = (1,2))
X_augmented[2::4] = rotate(X, angle = 180, axes = (1,2))
X_augmented[3::4] = rotate(X, angle = 270, axes = (1,2))

# one-hot encoding 
Y_augmented = np.zeros((X.shape[0]*4,4), dtype = int)
Y_augmented[::4,0] = 1
Y_augmented[1::4,1] = 1
Y_augmented[2::4,2] = 1
Y_augmented[3::4,3] = 1

## Train your model on pretext

In [ ]:
model2.fit(X_augmented,Y_augmented, batch_size = 128, epochs = 50, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule],validation_split = 0.3, shuffle = True)

Epoch 1/50
9/9 [==============================] - 58s 2s/step - loss: 1.6489 - accuracy: 0.2512 - val_loss: 1.2185 - val_accuracy: 0.3036
Epoch 2/50
9/9 [==============================] - 10s 1s/step - loss: 1.0624 - accuracy: 0.5308 - val_loss: 0.7588 - val_accuracy: 0.7455
Epoch 3/50
9/9 [==============================] - 10s 1s/step - loss: 0.5116 - accuracy: 0.8856 - val_loss: 0.3784 - val_accuracy: 0.9174
Epoch 4/50
9/9 [==============================] - 10s 1s/step - loss: 0.2332 - accuracy: 0.9221 - val_loss: 0.2354 - val_accuracy: 0.9353
Epoch 5/50
9/9 [==============================] - 10s 1s/step - loss: 0.1499 - accuracy: 0.9462 - val_loss: 0.2489 - val_accuracy: 0.9062
Epoch 6/50
9/9 [==============================] - 11s 1s/step - loss: 0.1307 - accuracy: 0.9505 - val_loss: 0.2021 - val_accuracy: 0.9375
Epoch 7/50
9/9 [==============================] - 11s 1s/step - loss: 0.0871 - accuracy: 0.9713 - val_loss: 0.2017 - val_accuracy: 0.9353
Epoch 8/50
9/9 [==================

# 5. Downstream task

## Train your model on downstream task

In [ ]:
model3, model_pretext3 = my_model_pretext(ishape = (250,250,3),k = 4, lr = 1e-6)
model3.load_weights(model_name_pretext)
model_pretext3.save_weights(model_name_pretext_no_top)

model3, model_pretext3 = my_model_pretext(ishape = (250,250,3),k = 4, lr = 1e-6)
model_pretext3.load_weights(model_name_pretext_no_top)
model_pretext3.trainable = False
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics = ["accuracy"])
print(model3.summary())

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 250, 250, 3)]     0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 250, 250, 48)      1344      
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 250, 250, 48)      20784     
_________________________________________________________________
dropout_18 (Dropout)         (None, 250, 250, 48)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 125, 125, 48)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 125, 125, 96)      41568     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 125, 125, 96)      830

In [ ]:

model3.fit(train_generator,batch_size = 128, epochs = 50, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule],validation_data=val_generator)

Epoch 1/10
98/98 [==============================] - 1446s 15s/step - loss: 1.3610 - accuracy: 0.4879 - val_loss: 0.6105 - val_accuracy: 0.7878
Epoch 2/10
98/98 [==============================] - 51s 524ms/step - loss: 0.5391 - accuracy: 0.8088 - val_loss: 0.5382 - val_accuracy: 0.7878
Epoch 3/10
98/98 [==============================] - 51s 528ms/step - loss: 0.4117 - accuracy: 0.8557 - val_loss: 0.4286 - val_accuracy: 0.8663
Epoch 4/10
98/98 [==============================] - 52s 526ms/step - loss: 0.3764 - accuracy: 0.8657 - val_loss: 0.3372 - val_accuracy: 0.9244
Epoch 5/10
98/98 [==============================] - 51s 525ms/step - loss: 0.3250 - accuracy: 0.8999 - val_loss: 0.3558 - val_accuracy: 0.8924
Epoch 6/10
98/98 [==============================] - 52s 528ms/step - loss: 0.3405 - accuracy: 0.8764 - val_loss: 0.3067 - val_accuracy: 0.9012
Epoch 7/10
98/98 [==============================] - 52s 526ms/step - loss: 0.3118 - accuracy: 0.8923 - val_loss: 0.3844 - val_accuracy: 0.8634

In [ ]:
model_pretext3.trainable = True
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model3.fit(train_generator,batch_size = 128, epochs = 50, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule],validation_data=val_generator)

Epoch 1/50
98/98 [==============================] - 62s 602ms/step - loss: 0.4838 - accuracy: 0.8394 - val_loss: 0.2037 - val_accuracy: 0.9506
Epoch 2/50
98/98 [==============================] - ETA: 0s - loss: 0.1844 - accuracy: 0.9425

KeyboardInterrupt: ignored

# 6. Test your model on the test and extract relevant metrics

In [ ]:
#using normal test generator 
model.load_weights(model_name_pretext)
metrics = model.evaluate(test_generator) # Get metrics from generators
print("Categorical cross-entropy:", metrics[0])
print("Accuracy:", metrics[1])

In [ ]:
#using generalized test set data
model.load_weights(model_name_pretext)
metrics = model.evaluate(general_test_generator)
print("Categorical cross-entropy:", metrics[0])
print("Accuracy:", metrics[1])